# Bundle adjustment
This notebook visualizes the optimization of 3D points and camera poses using bundle adjustment.

In [ ]:
from notebook_functions import (camera_centers, 
                                extrinsics, 
                                images, 
                                init_3d_plot, 
                                get_bunny, 
                                project_points_to_picture,
                                intrinsics, 
                                plot_camera_wireframe, 
                                plot_picture, 
                                update_camera_wireframe, 
                                update_picture, 
                                distort_extrinsics, 
                                camera_centers_from_extrinsics)
import ipyvolume as ipv
import numpy as np 
import time 
import g2o 

# Set up a scene and distort bunny
In this example, we will have perfect knowledge of the camera poses, perfect knowledge of observations, and extremely noisy estimates of 3D points. The goal is to recover the shape of the bunny through optimization.

In [ ]:
vis_scale = 5
init_3d_plot()
ipv.zlim(-35, 25)
bunny_coords = get_bunny() 
b_xs, b_ys, b_zs = bunny_coords[:3]
bunny_coords = np.vstack([bunny_coords, np.ones((1, bunny_coords.shape[1]))])
distort_bunny_coords = bunny_coords + np.random.random(bunny_coords.shape) * 5
b_xs, b_ys, b_zs, _ = distort_bunny_coords
scatter_plot = ipv.scatter(b_xs, b_ys, b_zs, size=0.5, marker="sphere", color='lime')
for cam_center, extrinsic, image in zip(camera_centers, extrinsics, images):
    image = np.zeros_like(image)
    image = project_points_to_picture(image, bunny_coords, intrinsics, extrinsic)
    inv_extrinsic = np.linalg.pinv(extrinsic)
    plot_camera_wireframe(cam_center, vis_scale, inv_extrinsic)
    plot_picture(image, inv_extrinsic, vis_scale)
    
ipv.show()

# Define cameras, points, and observations as a g2o graph
Here, we assume all points are visible in all cameras. As can be seen above, this is the case for this simple scene.

In [ ]:
# Sets optimizer 
optimizer = g2o.SparseOptimizer()
solver = g2o.BlockSolverSE3(g2o.LinearSolverCSparseSE3())
solver = g2o.OptimizationAlgorithmLevenberg(solver)
optimizer.set_algorithm(solver)

# Set real points in space
true_points = bunny_coords[:3]
distorted_points = distort_bunny_coords[:3]

# Set camera intrinsics
focal_length = (intrinsics[0, 0], intrinsics[1, 1])
principal_point = ((intrinsics[0, 2], intrinsics[1, 2]))
baseline = 0.0 # Not relevant, not a stereo setup
K = np.array([*focal_length, *principal_point, baseline])
g2o.VertexSCam.set_cam(*K)

# Create vertices from cameras and fix them, as their pose is known. 
for idx_extr, extrinsic in enumerate(extrinsics):
    pose = g2o.Isometry3d(np.linalg.inv(extrinsic))
    v_se3 = g2o.VertexSCam()
    v_se3.set_id(idx_extr)
    v_se3.set_estimate(pose)
    if idx_extr == 0:  # Fixing all cameras causes a segmentation fault, so unfix one
        v_se3.set_fixed(False)
    else:    
        v_se3.set_fixed(True)
    v_se3.set_all() # Calculates matrices related to projection
    optimizer.add_vertex(v_se3)

# Each vertex must have a unique id. 
# Ensure point and camera ids do not overlap.
last_pose_id = idx_extr
first_point_id = last_pose_id + 1

# Create vertices from all points and connect them to the cameras
for idx, (distorted_point, true_point) in enumerate(zip(distorted_points.T, true_points.T)):
    point_id = idx + first_point_id
    visible = []
    
    # Project the true points to the images, as observations are perfectly known
    for ext_id in range(len(extrinsics)):
        projected_point = optimizer.vertex(ext_id).map_point(true_point)
        visible.append((ext_id, projected_point))
    
    # Create a point vertex with distorted estimate
    vertex_point = g2o.VertexSBAPointXYZ()
    vertex_point.set_id(point_id)
    # see https://github.com/RainerKuemmerle/g2o/issues/109 on why we use this
    vertex_point.set_marginalized(True)  
    vertex_point.set_estimate(distorted_point)
    optimizer.add_vertex(vertex_point)
    
    # Create an point-camera edge with each camera
    for ext_idx, projected_point in visible:
        edge = g2o.Edge_XYZ_VSC()
        edge.set_vertex(0, vertex_point) 
        edge.set_vertex(1, optimizer.vertex(ext_idx)) 
        edge.set_measurement(projected_point) 
        # The information matrix is the measurement 
        # uncertainty. We set this equal for all measurements.
        edge.set_information(np.identity(3))
        edge.set_parameter_id(0, 0)
        optimizer.add_edge(edge)
    last_point_id = point_id

optimizer.initialize_optimization()
optimizer.set_verbose(False)
print("g2o graph has been created") 

# Visualize point optimization
Visualize 50 steps of point refinement using sparse bundle adjustment optimization.

In [ ]:
def vertex_to_points(optimizer, first_point_id, last_point_id): 
    """ Converts g2o point vertices to their current 3d point estimates. """
    vertices_dict = optimizer.vertices()
    estimated_points = list()
    for idx in range(first_point_id, last_point_id): 
        estimated_points.append(vertices_dict[idx].estimate())
    estimated_points = np.array(estimated_points)
    xs, ys, zs = estimated_points.T
    return xs, ys, zs 

ipv.show()
for i in range(100):
    optimizer.optimize(1)
    xs, ys, zs = vertex_to_points(optimizer, first_point_id, last_point_id)
    scatter_plot.x = xs 
    scatter_plot.y = ys 
    scatter_plot.z = zs 
    time.sleep(0.25)

 # Set up a scene and distort cameras

In [ ]:
def vertex_to_extrinsics(optimizer, last_pose_id):
    """ Converts g2o camera vertices to their current 3d camera pose estimates. """
    vertices_dict = optimizer.vertices()
    extrinsics = list()
    for idx in range(last_pose_id + 1): 
        extrinsic = np.linalg.inv(vertices_dict[idx].estimate().matrix())
        extrinsics.append(extrinsic)
    return extrinsics

distorted_extrinsics = [distort_extrinsics(extrinsic, max_angle=30, max_trans=5) for extrinsic in extrinsics]
distorted_extrinsics[0] = extrinsics[0] # Do not distort the first camera
distorted_centers = camera_centers_from_extrinsics(distorted_extrinsics)
vis_scale = 5
init_3d_plot()
ipv.zlim(-35, 25)


bunny_coords = get_bunny() 
b_xs, b_ys, b_zs = bunny_coords
bunny_coords = np.vstack([bunny_coords, np.ones((1, bunny_coords.shape[1]))])
scatter_plot = ipv.scatter(b_xs, b_ys, b_zs, size=0.5, marker="sphere", color='lime')

# Plot ground truth wireframes in red
for cam_center, extrinsic in zip(camera_centers, extrinsics):
    inv_extrinsic = np.linalg.pinv(extrinsic)
    plot_camera_wireframe(cam_center, vis_scale, inv_extrinsic, color='blue')

# Plot the distorted cameras  in blue
extrinsics_estimates = vertex_to_extrinsics(optimizer, last_pose_id)
old_plots = list()
for distort_center, distort_extrinsic, true_extrinsic in zip(distorted_centers, distorted_extrinsics, extrinsics):
    image = np.zeros_like(image)
    image = project_points_to_picture(image, bunny_coords, intrinsics, true_extrinsic) 
    inv_extrinsic = np.linalg.pinv(distort_extrinsic)
    wireframe = plot_camera_wireframe(distort_center, vis_scale, inv_extrinsic, color='red')
    picture = plot_picture(image, inv_extrinsic, vis_scale)
    old_plots.append([wireframe, picture])
    
ipv.show()


# Define cameras, points, and observations as a g2o graph

In [ ]:
optimizer = g2o.SparseOptimizer()
solver = g2o.BlockSolverSE3(g2o.LinearSolverCSparseSE3())
solver = g2o.OptimizationAlgorithmLevenberg(solver)
optimizer.set_algorithm(solver)

for idx_extr, extrinsic in enumerate(extrinsics):
    # Initialize with true intrisics so the 
    # projection images will be correct. 
    # Distorted extrinsic estimates are set at the end 
    pose = g2o.Isometry3d(np.linalg.inv(extrinsic))
    v_se3 = g2o.VertexSCam()
    v_se3.set_id(idx_extr)
    v_se3.set_estimate(pose)
    if idx_extr:
        v_se3.set_fixed(False)
    else: 
        # Fix the first camera
        v_se3.set_fixed(True)
    v_se3.set_all()
    optimizer.add_vertex(v_se3)
    
last_pose_id = idx_extr
first_point_id = last_pose_id + 1

for idx, true_point in enumerate(true_points.T):
    point_id = idx + first_point_id
    visible = []
    # Use true extrinsics to project points
    for ext_id in range(len(extrinsics)):
        projected_point = optimizer.vertex(ext_id).map_point(true_point)
        visible.append((ext_id, projected_point))
    vertex_point = g2o.VertexSBAPointXYZ()
    vertex_point.set_id(point_id)
    vertex_point.set_marginalized(True)
    vertex_point.set_estimate(true_point)
    optimizer.add_vertex(vertex_point)
    for ext_idx, projected_point in visible:
        edge = g2o.Edge_XYZ_VSC()
        edge.set_vertex(0, vertex_point) 
        edge.set_vertex(1, optimizer.vertex(ext_idx)) 
        edge.set_measurement(projected_point) 
        edge.set_information(np.identity(3))
        edge.set_parameter_id(0, 0)
        optimizer.add_edge(edge)
    last_point_id = point_id

# Set distorted extrinsics as pose estimate
for idx_extr, distorted_extrinsic in enumerate(distorted_extrinsics):
    if idx_extr: 
        pose = g2o.Isometry3d(np.linalg.inv(distorted_extrinsic))
        optimizer.vertex(idx_extr).set_estimate(pose)
        optimizer.vertex(idx_extr).set_all()

optimizer.initialize_optimization()
optimizer.set_verbose(False)
print("g2o graph has been created") 

# Visualize camera pose optimization
Visualize 400 steps of camera pose refinement using sparse bundle adjustment optimization.

In [ ]:
ipv.show()
time.sleep(2)
for _ in range(50):
    optimizer.optimize(2)
    extrinsics_estimates = vertex_to_extrinsics(optimizer, last_pose_id)
    new_plots = list()
    for idx, (extrinsic, image) in enumerate(zip(extrinsics_estimates, images)):
        wireframe, picture = old_plots[idx]
        cam_center = -extrinsic[:3, :3].T @ extrinsic[:3, 3]
        inv_extrinsic = np.linalg.pinv(extrinsic)
        wireframe = update_camera_wireframe(cam_center, vis_scale, inv_extrinsic, wireframe)
        new_picture = update_picture(image.copy(), inv_extrinsic.copy(), vis_scale, picture)
        new_plots.append([wireframe, new_picture])
    old_plots = new_plots
    time.sleep(2)